In [1]:
%load_ext autoreload
%autoreload 2

import pennylane as qml
import math
from pennylane import qaoa
from pennylane import numpy as np
import matplotlib.pyplot as plt
from vqa.hamiltonian.protein_folding import protein_folding_hamiltonian
from vqa.hamiltonian.Protein_models.CoordinateBased_HPLattice import CoordinateBased_HPLattice
from vqa.utils.protein_utils import *
from collections import Counter
import pandas as pd
import csv
import os.path

In [2]:
# Make an instance for the protein
sequence = [1,0,0,1] #HPPH
L1 = 3
L2 = 2

lambda_vector = (2.1, 2.4, 3)

protein = CoordinateBased_HPLattice((L1, L2), sequence = sequence, lambda_vector = lambda_vector)
print(protein)
protein.calc_solution_sets()
feasible_set = np.array(protein.feasible_set)


O:
[-2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1]
T:
[[ 0.   4.2  4.2  0.   0.   3.   2.4  0.   0.  -1.  -1.   0. ]
 [ 4.2  0.   4.2  0.   0.   0.   0.   2.4  0.  -1.  -1.  -1. ]
 [ 4.2  4.2  0.   3.   0.   0.   0.   0.   2.4  0.  -1.  -1. ]
 [ 0.   0.   3.   0.   4.2  4.2  0.   0.   3.   2.4  0.   0. ]
 [ 0.   0.   0.   4.2  0.   4.2  0.   0.   0.   0.   2.4  0. ]
 [ 3.   0.   0.   4.2  4.2  0.   3.   0.   0.   0.   0.   2.4]
 [ 2.4  0.   0.   0.   0.   3.   0.   4.2  4.2  0.   0.   3. ]
 [ 0.   2.4  0.   0.   0.   0.   4.2  0.   4.2  0.   0.   0. ]
 [ 0.   0.   2.4  3.   0.   0.   4.2  4.2  0.   3.   0.   0. ]
 [-1.  -1.   0.   2.4  0.   0.   0.   0.   3.   0.   4.2  4.2]
 [-1.  -1.  -1.   0.   2.4  0.   0.   0.   0.   4.2  0.   4.2]
 [ 0.  -1.  -1.   0.   0.   2.4  3.   0.   0.   4.2  4.2  0. ]]
Dn:
[3, 3, 3, 3]


In [3]:
# Make into Hamiltonian
H_cost = protein_folding_hamiltonian(protein)
num_qubits = protein.num_bits
qubits = range(protein.num_bits)

H_mixer = qaoa.x_mixer(qubits)

In [4]:
dev = qml.device('default.qubit', wires = qubits)

# Put the Mixer and Cost Hamiltonians in a layer so that we can repeat it
def qaoa_layer(gamma, beta):
    qaoa.cost_layer(gamma, H_cost)
    qaoa.mixer_layer(beta, H_mixer)
    
# Then repeat it in a circuit with starting in a super position of all bitstrings
def circuit(params):     # Gamma and Beta values can be put together to be an array of parameters
    for q in qubits:     # To start in a superposition we place a Hadamard on all qubits
        qml.Hadamard(wires = q)
    qml.layer(qaoa_layer, len(params[0]), params[0], params[1])
    
# Obtains the probability for all the states
@qml.qnode(dev)
def get_probs(params):
    circuit(params)
    return qml.probs(wires = qubits)

# CVaR function for known energies and probabilities
def CVaR(params, alpha=0.25):
    circuit(params)
    
    Energy_list = get_energies_index_states(H_cost)
    Energy_array = np.column_stack((Energy_list, get_probs(params)))
    Energy_array = Energy_array[Energy_array[:, 0].argsort()]
    
    Probability_array = np.array(Energy_array[0:len(Energy_list),1])
    Probability_vector = Probability_array[Probability_array.cumsum() <= alpha]
    
    K = len(Probability_vector)
    Energy_vector = Energy_array[0:K,0]
    exp_CVaR = np.dot(Energy_vector, Probability_vector)/np.sum(Probability_vector)
    
    return exp_CVaR

def CVaRsamp(params, alpha, n):#n=antal samples
    circuit(params)
    
    probs = get_probs(params)  # hämta sannolikhetsfördelningen (matris, första värdet är p(00000000), andra är p(00000001) osv)
    
    index_samples= np.random.choice(np.arange(len(probs), dtype=int), size=n, replace=True,p=probs)  # tar n samples av probs, ger skum lista med index motsvarande konfiguration (index 0 är tillståndet 00000000, index 1 =00000001 osv)
    energy_of_samples=energies_of_set(protein.get_solution_set(), H_cost,8) [index_samples]  #ger en lista där index i index_samples är utbytta mot deras motsvarande energi. 
    sorted_energy_samples=sort_over_threshhold(energy_of_samples,-10)[0] #sorterar hela energilistan
    K=int(alpha*n) #antal samples att ta väntevärde över. 
    summa=sum(sorted_energy_samples[:K]) #summera de K minsta energierna.
    expvalue=np.float64(summa/K)
    
    return expvalue

In [10]:
#OBS: se till att du har GPyOpt och pyDOE installerat på datorn ( pip install GPyOpt och pip install pyDOE)

import GPyOpt
from GPyOpt.methods import BayesianOptimization

#@qml.qnode(dev)
#def probability_circuit(params):
#    circuit(params)
#    return qml.probs(wires=qubits)

#probs = probability_circuit(params)

#def success_prob(params):
#    ground_energy, ground_states_i = get_ground_states_i(feasible_set, H_cost)
#    return np.sum(probs[ground_states_i])


def trainGPyOpt( 
			stepsize = 0.01,
			steps = 100,
			heuristic = 'CVaR',
			n = None,
			swarm = 20,
			initial_design_type = 'latin',
			acquisition_type='EI',
			exact_feval = False,
			given_init_params = np.full((2, 1), None),
			plot = False,
			verbose = False,
			gamma_min = 0,
			gamma_max = np.pi*2,
			beta_min = 0,
			beta_max = np.pi*2
			):
	'''
	swarm = 0 and given init params will not swarm. 
	'''

	def wrap_cost(params_list):
		if heuristic == 'CVaR':
			cost_function = CVaRsamp
		elif heuristic == 'success prob':
			cost_function = neg_success_prob   	# we want to maximise the probability
		elif heuristic == 'n-samples':
			n = n
			cost_function = n_samples

		gamma_list = params_list[0][:p]
		beta_list = params_list[0][p:]
		params = np.array([gamma_list, beta_list])
		return cost_function(params, alpha, num_of_samples)

	#t0 = time.time()

	gamma = []
	beta = []


	for i in range(p):
		gamma += [{'name': 'gamma'+str(i+1),
					'type': 'continuous', 'domain': (gamma_min, gamma_max)}]
		beta += [{'name': 'beta'+str(i+1),
					'type': 'continuous', 'domain': (beta_min, beta_max)}]
	bounds = gamma + beta
	myBopt = GPyOpt.methods.BayesianOptimization(wrap_cost,
													domain=bounds,
													initial_design_numdata = swarm,
													verbosity = verbose,
													initial_design_type = initial_design_type,
													acquisition_type = acquisition_type,
													exact_feval = exact_feval)
	if swarm == 0 and given_init_params.all() != None:
		print('Given init')
		myBopt.X = np.array([given_init_params.flatten().tolist()])
		myBopt.Y = wrap_cost(np.array([given_init_params.flatten().tolist()])).reshape(-1, 1)

	myBopt.run_optimization(max_iter = steps, verbosity = verbose)
	if p == 1:
		pass #myBopt.plot_acquisition()
		#plt.savefig('acq.pdf')

	

	if plot:
		myBopt.plot_convergence()
		plt.savefig('conv.pdf')
		plot_cost(cost_vector, new_fig = False, save = True, name = heuristic)


	best_params = np.array([myBopt.x_opt[:p], myBopt.x_opt[p:]])
	params_vector, cost_vector = myBopt.get_evaluations()
	#t1 = time.time()

	#print("="*20)
	#print("Value of (gamma, beta) that minimises the objective:"+str(myBopt.x_opt))    
	#print("Minimum value of the objective: "+str(myBopt.fx_opt))   
	#print("="*20)

	return best_params, cost_vector, params_vector

In [13]:
@qml.qnode(dev)
def probability_circuit(params):
    circuit(params)
    return qml.probs(wires=qubits)

p = 2
alpha = 0.1
m = [1000]
     #12000,23000,34000,45000,56000,67000,78000,89000,100000]
#a = [.1, .15, .20, .25, .30, .35, .4, .45, .5, .55, .6, .65, .7, .75, .80, .85, .9, .95, 1]
for num_of_samples in m:
    best_params, cost_vector, params_vector = trainGPyOpt()

    probs = probability_circuit(best_params)

    ground_energy, ground_states_i = get_ground_states_i(feasible_set, H_cost) # get the ground states
    
    print('Alpha (α) = ' + str(alpha))
    print('Number of samples is: ' + str(m))
    print('Success probability of training: ', np.sum(probs[ground_states_i]))
    plot_probs_with_energy(probs, num_qubits, H_cost, ground_states_i) # plot probability over bitstrings

Alpha (α) = 0.1
Number of samples is: [1000]
Success probability of training:  0.004381989546192648


ZeroDivisionError: float division by zero

Error in callback <function _draw_all_if_interactive at 0x7f1226c1cd30> (for post_execute):


ValueError: cannot convert float NaN to integer

<Figure size 640x480 with 0 Axes>

ValueError: cannot convert float NaN to integer

<Figure size 1500x700 with 2 Axes>

In [92]:
@qml.qnode(dev)
def probability_circuit(params):
    circuit(params)
    return qml.probs(wires=qubits)

p = 3
alpha = 0.1
m = [20, 40, 60, 80, 100]
     #200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500]

for num_of_samples in m:
    counter = 0
    j = 0
    for i in range(50):
        j=i+1
        best_params, cost_vector, params_vector = trainGPyOpt()

        probs = probability_circuit(best_params)

        ground_energy, ground_states_i = get_ground_states_i(feasible_set, H_cost) # get the ground states

        if np.sum(probs[ground_states_i]) < 0.05:
            counter+=1
        #if counter == 2:
            #break
        print('Try ' + str(j) + ': Success probability of training: ', np.sum(probs[ground_states_i]))
    print('For: ' + str(num_of_samples) + ' samples, the prob of measuring any groundstate was at least 5% : ' + str(j-counter) + ' times out of ' + str(j) + ' tries.')
    print("="*20)
    

Try 1: Success probability of training:  0.09823831505046958


KeyboardInterrupt: 

In [32]:
#binary search för att minimera nr_of_samps

@qml.qnode(dev)
def probability_circuit(params):
    circuit(params)
    return qml.probs(wires=qubits)

def test2(): # kör p=1 först
    counter = 0
    j = 0
    for i in range(50):
        j=i+1
        best_params, cost_vector, params_vector = trainGPyOpt(stepsize=stegsize, steps=steg, swarm=sverm)

        probs = probability_circuit(best_params)

        ground_energy, ground_states_i = get_ground_states_i(feasible_set, H_cost) # get the ground states

        if np.sum(probs[ground_states_i]) < 0.05:
            counter+=1
        if counter == 6:
            return False
        print('Try ' + str(j) + ': Success probability of training: ', np.sum(probs[ground_states_i]))
    print('For: ' + str(num_of_samples) + ' samples, the prob of measuring any groundstate was at least 5% : ' + str(j-counter) + ' times out of ' + str(j) + ' tries.')
    print("="*20)
    return True

p=1
sverm=1
stegsize=0.05
steg=150
alpha=0.7 #  o 0.5
lo=10
hi =150

while lo <= hi:
 
    num_of_samples = (hi + lo) // 2
    # print(str(mid))
     
    # If x is greater, ignore left half
    if test2():
        hi = num_of_samples +1 # +1 och -1 behövs nog inte
        print('sant för ' + str(num_of_samples) + ' går till ' + str((hi+lo)//2))
    else:
        lo = num_of_samples -1
        print('falskt för ' +str(num_of_samples) + ' går till ' + str((hi+lo)//2))
        
    
    if abs(lo-hi) < 5: #hur nära rätt ska de va?
        bestsamp=hi-1
        print(bestsamp)
        break
#spika steps = 10 eller ngt, sätt lo o hi = sampantal som du vet inte ger rätt respektive ger rätt. kör mid. om fel,
#sätt lo=mid. om rätt, sätt hi=mid. kör om. slut när skillnaden mellan lo o hi når ngt threshold, då kan man ta hi bara.  
#detta ger bäst samp för specifikt steps. fkn, skulle vilja skicka in ett värde på samp, och få ut om de funkade eller ej.



Try 1: Success probability of training:  0.19498415736838348
Try 2: Success probability of training:  0.19596722332706137
Try 3: Success probability of training:  0.1883161952733053
Try 4: Success probability of training:  0.19644687150936377
Try 5: Success probability of training:  0.10799279763714872
Try 6: Success probability of training:  0.19441292411842395
Try 7: Success probability of training:  0.19232891508039868
Try 8: Success probability of training:  0.1415468140556867
Try 9: Success probability of training:  0.17797660250290961
Try 10: Success probability of training:  0.1975155756868103
Try 11: Success probability of training:  0.19206884991798712
Try 12: Success probability of training:  0.08655123730631614
Try 13: Success probability of training:  0.18831840279055986
Try 14: Success probability of training:  0.19353674089635153
Try 15: Success probability of training:  0.1775725950767247
Try 16: Success probability of training:  0.1894004452165221
Try 17: Success probab

Try 40: Success probability of training:  0.18777426843301886
Try 41: Success probability of training:  0.10410032928643008
Try 42: Success probability of training:  0.1954676737899718
Try 43: Success probability of training:  0.011214571754763487
Try 44: Success probability of training:  0.11058996765767523
Try 45: Success probability of training:  0.09004967038422775
Try 46: Success probability of training:  0.17604610140675292
Try 47: Success probability of training:  0.07915456001567099
Try 48: Success probability of training:  0.18841557867860753
Try 49: Success probability of training:  0.1107141221826826
Try 50: Success probability of training:  0.07179302699349072
For: 71 samples, the prob of measuring any groundstate was at least 5% : 45 times out of 50 tries.
sant för 71 går till 66
Try 1: Success probability of training:  0.0029165779872566376
Try 2: Success probability of training:  0.061608222532469295
Try 3: Success probability of training:  0.1839112759976933
Try 4: Succ

Try 41: Success probability of training:  0.11296856363667185
Try 42: Success probability of training:  0.10985897614469364
Try 43: Success probability of training:  0.08485662315472522
falskt för 69 går till 70
71


In [30]:
#kod med filsparning och valda nr_of_samps

@qml.qnode(dev)
def probability_circuit(params):
    circuit(params)
    return qml.probs(wires=qubits)

def test3(): # kör p=1 först
    counter = 0
    j = 0
    for i in range(50):
        j=i+1
        best_params, cost_vector, params_vector = trainGPyOpt(stepsize=stegsize, steps=steg, swarm=sverm)

        probs = probability_circuit(best_params)

        ground_energy, ground_states_i = get_ground_states_i(feasible_set, H_cost) # get the ground states

        if np.sum(probs[ground_states_i]) < 0.05:
            counter+=1
        p_groundstate=np.sum(probs[ground_states_i])
        datalista.append([p_groundstate, num_of_samples])
        print('Try ' + str(j) + ': Success probability of training: ', p_groundstate)
    datalista.append(['prob of meas any groundstate was >5% this # times: ',str(j-counter)+ '/' +str(j)])
    print('For: ' + str(num_of_samples) + ' samples, the prob of measuring any groundstate was at least 5% : ' + str(j-counter) + ' times out of ' + str(j) + ' tries.')
    print("="*20)

#parametrar
steg=150
stegsize=0.05
sverm=1
p=1
alpha=0.5


datalista=[] #lista, görs till pandasdatabas och sedan till csv. 
m=[40,50]#vilka num of samps ska köras?

for num_of_samples in m: 
    test3()

print(datalista)

#nedan är bara datasparandet, ta bort om du vill
sek=''
for i in sequence: #omvanla 1001 till HPPH
    if i==1: 
        sek+='H'
    else: 
        sek+='P'
        
parms=('p: ' + str(p) + ', alpha: ' + str(alpha) + 
       ', steps:  ' + str(steg) + ', stepsize: ' + str(stegsize) + 
       ', sekvens: ' + str(sek) + ', gridsize: ' + str(L1)+'X'+str(L2)) #filens titel

#kolla om fil redan finns, isf lägg till 1 i slutet av filnamnet 8)
while os.path.isfile('pettson_data_test/'+str(sek)+'/'+ str(L1)+'X'+str(L2)+'/' + str(parms) + '.csv'):
    if parms[-1]!=1:
        parms+=(':1')
    else:
        parms[-1]+=1
        
df=pd.DataFrame(datalista) # gör lista till pandas dataframe

df.to_csv('pettson_data_test/'+str(sek)+'/'
          + str(L1)+'X'+str(L2)+'/' + str(parms) + '.csv', 
          index=False, header = False) #spara i rätt mapp (pettson -> sekvens -> 2X2 )

     


Try 1: Success probability of training:  0.007711819763804109
Try 2: Success probability of training:  0.08222519285174242
For: 40 samples, the prob of measuring any groundstate was at least 5% : 1 times out of 2 tries.
Try 1: Success probability of training:  0.0017269849538847286
Try 2: Success probability of training:  0.08802942837304074
For: 50 samples, the prob of measuring any groundstate was at least 5% : 1 times out of 2 tries.
[[tensor(0.00771182, requires_grad=True), 40], [tensor(0.08222519, requires_grad=True), 40], ['prob of meas any groundstate was >5% this # times: ', '1/2'], [tensor(0.00172698, requires_grad=True), 50], [tensor(0.08802943, requires_grad=True), 50], ['prob of meas any groundstate was >5% this # times: ', '1/2']]


['p: 1, alpha: 0.5, steps:  20, stepsize: 0.05, sekvens: HPPH, gridsize: 2X2', '1', '1']
<class 'list'>
